In [2]:
import torch
torch.cuda.is_available()
print(torch.__version__)
torch.cuda.get_device_name(0)

1.1.0


'GeForce RTX 2070'

In [2]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path


corpus = NLPTaskDataFetcher.load_classification_corpus(Path('flair_data/'), test_file = 'test.csv', dev_file = 'dev.csv', train_file = 'train.csv')#.downsample(0.01)
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size = 512, reproject_words = True, reproject_words_dimension = 256)
classifier = TextClassifier(document_embeddings, label_dictionary = corpus.make_label_dictionary(), multi_label = True)
trainer = ModelTrainer(classifier, corpus)

2019-05-09 13:13:56,886 Reading data from flair_data
2019-05-09 13:13:56,888 Train: flair_data/train.csv
2019-05-09 13:13:56,888 Dev: flair_data/dev.csv
2019-05-09 13:13:56,889 Test: flair_data/test.csv


/Users/davidmortensen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
trainer.train('./', max_epochs=10, mini_batch_size = 8)

2019-05-09 13:23:21,214 ----------------------------------------------------------------------------------------------------
2019-05-09 13:23:21,215 Evaluation method: MICRO_F1_SCORE
2019-05-09 13:23:21,220 ----------------------------------------------------------------------------------------------------
2019-05-09 13:23:29,250 epoch 1 - iter 0/90 - loss 0.09685771
2019-05-09 13:24:57,747 epoch 1 - iter 9/90 - loss 0.09010749
